In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install pandas

In [1]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains 
import re

# Парсинг данных

In [2]:
driver = webdriver.Chrome()
driver.get('https://cryptobriefing.com/news/')

data = []

while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    containers = soup.find_all("li",attrs={"class":"main-news-item"})

    for i, d in enumerate(containers):
        #print(f"Analyzing containers. i = {i}")
        if i < len(data) - 1:
            continue
        d = d.find('section')
        d = d.find('div', {'class': "main-news-info"})

        header = d.find("h2", {"class": "main-news-title"})
        header = header.find("a", {"class": "main-news-link"})
        if header is None:
            header_text = "NOT FOUND"
            href = "NOT FOUND"
            #category = "NOT FOUND"
        else:
            header_text = header['aria-label'].strip()
            href = header['href'].strip()
            #category = header.text.strip()
        
        meta = d.find("p", {"class": "main-news-meta"})
        time_meta = meta.find("time")
        meta = meta.find("a", {"class": "main-news-category-link"})

        if meta is None:
            category = "NOT FOUND"
            dttime = "NOT FOUND"
        else:
            category = meta.text.strip()

        if time_meta is None:
            dttime = "NOT FOUND"
        else:
            dttime = time_meta['datetime']
            if dttime is None:
                dttime = time_meta['title']

        msg = d.find("p", {"class": "main-news-message"})
        if msg is None:
            brief = "NOT FOUND"
        else:
            brief = msg.text.strip()

        print(i, header_text, dttime, href, category, brief)
        data.append((header_text, dttime, href, category, brief)) 

        #data.append((header, timedata, ref))
        #print(f"Found header: {header}, date: {timedata}, ref: {ref}")

    button = driver.find_element(By.XPATH, '/html/body/main/div/div/div/section/div/a')
    location = button.location
    size = button.size
    w, h = size['width'], size['height']
    driver.execute_script(f"window.scrollTo(0, {h});")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(1)


0 Open Trump to host exclusive dinner for top $TRUMP holders, token jumps over 70% 2025-04-23T17:45:27+00:00 https://cryptobriefing.com/exclusive-trump-dinner-for-token-holders/ Meme Coins The event highlights the influence of celebrity endorsements on cryptocurrency markets, raising questions about financial ethics and regulatory oversight.
1 Open Cantor-backed Bitcoin venture launches with goal to amass 42,000 BTC 2025-04-23T16:17:54+00:00 https://cryptobriefing.com/bitcoin-venture-launch-42000-btc/ Business The launch of Twenty One Capital could accelerate Bitcoin's institutional adoption, potentially reshaping global financial systems.
2 Open Bitcoin flips Google and silver to become the 5th most valuable global asset 2025-04-23T13:30:32+00:00 https://cryptobriefing.com/bitcoin-overtakes-google-top-asset/ Bitcoin Bitcoin's rise highlights its growing role as a hedge against economic uncertainty, potentially reshaping global asset dynamics and investment strategies.
3 Open Bitcoin e

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=135.0.7049.114)
Stacktrace:
	GetHandleVerifier [0x00007FF634FDEFA5+77893]
	GetHandleVerifier [0x00007FF634FDF000+77984]
	(No symbol) [0x00007FF634DA91BA]
	(No symbol) [0x00007FF634D94F15]
	(No symbol) [0x00007FF634DB9F04]
	(No symbol) [0x00007FF634E2EACF]
	(No symbol) [0x00007FF634E4E972]
	(No symbol) [0x00007FF634E26F03]
	(No symbol) [0x00007FF634DF0328]
	(No symbol) [0x00007FF634DF1093]
	GetHandleVerifier [0x00007FF635297B6D+2931725]
	GetHandleVerifier [0x00007FF635292132+2908626]
	GetHandleVerifier [0x00007FF6352B00F3+3031443]
	GetHandleVerifier [0x00007FF634FF91EA+184970]
	GetHandleVerifier [0x00007FF63500086F+215311]
	GetHandleVerifier [0x00007FF634FE6EC4+110436]
	GetHandleVerifier [0x00007FF634FE7072+110866]
	GetHandleVerifier [0x00007FF634FCD479+5401]
	BaseThreadInitThunk [0x00007FF821A27374+20]
	Rtl

In [3]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link', 'category', 'brief'])
df = df.drop_duplicates()
df.tail()

,header,date,link,category,brief
10876,Open What Is Loom Network? Introduction to LO...,2019-04-03T23:42:47+00:00,https://cryptobriefing.com/what-is-loom-networ...,Education,The Loom Network is an Ethereum-based sidechai...
10877,Open Put Your Crypto To Work: Five Ways To Ear...,2019-04-03T22:08:03+00:00,https://cryptobriefing.com/five-ways-crypto-pa...,Education,You don't need to mine to earn cryptocurrencie...
10878,Open Nash Exchange: The Compliant Crypto DEX O...,2019-04-03T21:07:47+00:00,https://cryptobriefing.com/nash-exchange-crypt...,Business,Nash Exchange is a highly-anticipated non-cust...
10879,Open Smartlands Steps Closer To A Regulated Se...,2019-04-03T19:34:21+00:00,https://cryptobriefing.com/smartlands-steps-cl...,Business,Security token platform Smartlands is making s...
10880,Open Analysis: Will Bitcoin Jump Again?,2019-04-03T18:59:53+00:00,https://cryptobriefing.com/will-bitcoin-jump-a...,Analysis,Judging by price movements in the past six mon...


In [4]:
df.to_csv("../data/cryptobriefing_2025_2019_04_03.csv", index=False)